<a href="https://colab.research.google.com/github/swordman08/DataScienceProjects/blob/main/NLP_CharacterPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import re
import requests
import io
# book we are using (utf-8 format) https://www.gutenberg.org/cache/epub/2600/pg2600.txt

In [ ]:
# URL of the text file
url = 'https://www.gutenberg.org/cache/epub/2600/pg2600.txt'

try:
    # Fetch the file
    response = requests.get(url)
    response.raise_for_status()  # Check if request was successful

    # Decode the content with UTF-8 and load into a file-like object
    text_file = io.StringIO(response.content.decode('utf-8'))

    # Read the content
    text1 = text_file.read().lower()

    text = text1[8300:]


    # Define the number of characters you want to remove from the end
    x = 18830  # characters to remove off end

    # Remove the last x characters
    trimmed_text = text[:-x] if x < len(text) else ""
    trimmed_part = text[-x:] if x < len(text) else text  # The part that was removed

    # Print or process the trimmed part and the remaining text
    print("Trimmed Part (last", x, "characters):\n", trimmed_part)


    # Print or use the trimmed text


    # Step 1: Data Preparation


    # Basic text cleaning to remove unwanted characters
    # We’re keeping only letters and basic punctuation for simplicity

    text = re.sub(r'[^a-zA-Z ,.!?]', '', trimmed_text)

     # Get the file size in bytes, each Character depending on if it sspecial
    file_size_bytes = len(response.content)

    # megabyte = 1,048,576 bytes.
    file_size_mb = file_size_bytes / (1024 ** 2)

    # Print file size in MB and Bytes
    # each character has on average 1 byte, but special characters and symbols can have up to 4. This is just for UTF-8 encoded text.
    print(f"File size: {file_size_mb:.2f} MB")
    print(f"File size: {file_size_bytes:,} bytes")

    # Process or print content
    # print(content)

except requests.exceptions.RequestException as e:
    print("Error accessing the file:", e)





Trimmed Part (last 18830 characters):
 ** end of the project gutenberg ebook war and peace ***


    

updated editions will replace the previous one—the old editions will
be renamed.

creating the works from print editions not protected by u.s. copyright
law means that no one owns a united states copyright in these works,
so the foundation (and you!) can copy and distribute it in the united
states without permission and without paying copyright
royalties. special rules, set forth in the general terms of use part
of this license, apply to copying and distributing project
gutenberg™ electronic works to protect the project gutenberg™
concept and trademark. project gutenberg is a registered trademark,
and may not be used if you charge for an ebook, except by following
the terms of the trademark license, including paying royalties for use
of the project gutenberg trademark. if you do not charge anything for
copies of this ebook, complying with the trademark license is very
easy. you may us

In [ ]:


# Print or process `text` to verify
print(text[:500])  # Print the first 500 characters to verify

# Now, you can proceed to tokenize and prepare sequences with `text`

well, prince, so genoa and lucca are now just family estates of thebuonapartes. but i warn you, if you dont tell me that this means war,if you still try to defend the infamies and horrors perpetrated by thatantichristi really believe he is antichristi will have nothingmore to do with you and you are no longer my friend, no longer myfaithful slave, as you call yourself! but how do you do? i see ihave frightened yousit down and tell me all the news.it was in july, , and the speaker was the wellkno


In [ ]:
# Step 2: Character Mapping

# Create a sorted list of unique characters in the text.
# We do this because computers cannot understand language, only numbers.
chars = sorted(list(set(text)))
char_to_int = {char: i for i, char in enumerate(chars)}  # map characters to integers
int_to_char = {i: char for i, char in enumerate(chars)}  # map integers back to characters

# Display the mappings to understand the character encoding
print(f"Unique Characters: {chars}")
print(f"Character to Integer Mapping: {char_to_int}")


Unique Characters: [' ', '!', ',', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Character to Integer Mapping: {' ': 0, '!': 1, ',': 2, '.': 3, '?': 4, 'a': 5, 'b': 6, 'c': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'j': 14, 'k': 15, 'l': 16, 'm': 17, 'n': 18, 'o': 19, 'p': 20, 'q': 21, 'r': 22, 's': 23, 't': 24, 'u': 25, 'v': 26, 'w': 27, 'x': 28, 'y': 29, 'z': 30}


In [ ]:
# Step 3: Sequence Creation

# Sequence length, i.e., how many characters the model will look back to predict the next one
sequence_length = 150
sequences = []
next_chars = []



# Loop to create sequences and the corresponding target character
for i in range(0, len(text) - sequence_length):
    sequences.append(text[i:i + sequence_length])
    next_chars.append(text[i + sequence_length])

print(f"Total Sequences Created: {len(sequences)}")
# total sequences calculated as follows ,      (number of total character - sequence length) / stride



# Convert the characters in sequences and next_chars to integers
X = np.zeros((len(sequences), sequence_length), dtype=np.int32)
y = np.zeros((len(sequences)), dtype=np.int32)


for i, seq in enumerate(sequences):
    X[i] = [char_to_int[char] for char in seq]
    y[i] = char_to_int[next_chars[i]]

# Convert y to categorical format for prediction
y = to_categorical(y, num_classes=len(chars))



Total Sequences Created: 3092071


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Initialize an enhanced sequential model
model = Sequential()





# 1. Embedding layer to learn representations of characters
model.add(Embedding(input_dim=len(chars), output_dim=128, input_length=sequence_length))

# 2. First Bidirectional LSTM layer with 256 units
# Bidirectional LSTM helps capture context in both forward and backward directions
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.3))  # Dropout to reduce overfitting

# 3. Batch normalization for stable training
model.add(BatchNormalization())

# 4. Second Bidirectional LSTM layer with 512 units for deeper learning
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# 5. Third LSTM layer without Bidirectional; fewer units (e.g., 256) to condense information
model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.3))

# 6. Additional Dense layer to capture more complex relationships after LSTM layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# 7. Output Dense layer with softmax activation for character prediction
model.add(Dense(len(chars), activation='softmax'))

# Compile the model with a clipped Adam optimizer and a learning rate scheduler
model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler to reduce learning rate if there's no improvement
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-5)

# Display the model architecture for review
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Adjust callbacks to monitor validation loss
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=2,  # Slightly increase patience to avoid early stopping too soon
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,
    patience=2,
    min_lr=1e-5,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',  # Monitor validation loss
    save_best_only=True,
    verbose=1
)



import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# Convert y to categorical labels
# y = to_categorical(y, num_classes=len(chars))

# Split data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display shapes for verification
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

# Now you can train the model with the 80-20 split\

# Increased patience to allow for small fluctuations in validation loss
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)


model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=10,
    validation_data=(X_test, y_test),  # Validate on the 20% test split
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# accuracy far better than random (3.23%) means we have a working model.
# (number of predicted chars (1) / number of total unique character choices (31) = 1/31 = 3.23%)
# Even an accuracy of 50% would mean we are only wrong half the time,
# as opposed to being wrong 30 out of 31 times with random accuracy.



Training data shape: (2473596, 150) (2473596, 31)
Testing data shape: (618400, 150) (618400, 31)
Epoch 1/10
38650/38650 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4100 - loss: 2.0063
Epoch 1: val_loss improved from inf to 1.52762, saving model to best_model.keras
38650/38650 ━━━━━━━━━━━━━━━━━━━━ 1801s 47ms/step - accuracy: 0.4100 - loss: 2.0063 - val_accuracy: 0.5345 - val_loss: 1.5276 - learning_rate: 0.0010
Epoch 2/10
38650/38650 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5135 - loss: 1.6277
Epoch 2: val_loss improved from 1.52762 to 1.45139, saving model to best_model.keras
38650/38650 ━━━━━━━━━━━━━━━━━━━━ 1802s 47ms/step - accuracy: 0.5135 - loss: 1.6277 - val_accuracy: 0.5570 - val_loss: 1.4514 - learning_rate: 0.0010
Epoch 3/10
38649/38650 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5309 - loss: 1.5643
Epoch 3: val_loss improved from 1.45139 to 1.43819, saving model to best_model.keras
38650/38650 ━━━━━━━━━━━━━━━━━━━━ 1803s 47ms/step - accuracy: 0.5309 - loss: 1.564

In [ ]:
from joblib import Parallel, delayed
import joblib


# Save the model as a pickle in a file
joblib.dump(model, 'best_model_LSTM.pkl')

# Load the model from the file
model_LSTM = joblib.load('best_model_LSTM.pkl')

# Use the loaded model to make predictions
#knn_from_joblib.predict(X_test)

In [ ]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 150, 128)            │           3,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 150, 512)            │         788,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 150, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 150, 512)            │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ (None, 150, 1024)           │       4,198,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 150, 1024)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 150, 1024)           │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 256)                 │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 31)                  │           7,967 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,141,343 (73.02 MB)

 Trainable params: 6,379,423 (24.34 MB)

 Non-trainable params: 3,072 (12.00 KB)

 Optimizer params: 12,758,848 (48.67 MB)

In [ ]:
import numpy as np

def generate_text(model, seed_text, num_chars=60, temperature=0.7):
    generated_text = seed_text
    for _ in range(num_chars):
        input_seq = np.array([char_to_int[char] for char in seed_text]).reshape(1, -1)
        predictions = model.predict(input_seq, verbose=0)[0]

        # Adjust predictions by temperature
        predictions = np.log(predictions + 1e-10) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))

        # Sample the next character based on adjusted probabilities
        next_index = np.random.choice(len(predictions), p=predictions)
        next_char = int_to_char[next_index]

        generated_text += next_char
        seed_text = seed_text[1:] + next_char

    return generated_text







# Testing the generation function with a seed text
seed_text = text[:sequence_length]  # Starting text to kick off the prediction
generated_text = generate_text(model, seed_text)
print("Generated Text:")
print(generated_text)

KeyError: 'j'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split

# Assuming `text` is your 250,000 character sample
text = text[:750000]
# Encode the text
unique_chars = sorted(set(text))
char_to_int = {char: idx for idx, char in enumerate(unique_chars)}
encoded_text = [char_to_int[char] for char in text]

# Prepare sequences
seq_length = 50
X_data, y_data = [], []
for i in range(len(encoded_text) - seq_length):
    X_data.append(encoded_text[i:i + seq_length])
    y_data.append(encoded_text[i + seq_length])

X_data, y_data = np.array(X_data), np.array(y_data)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Build a more complex model
vocab_size = len(unique_chars)
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=seq_length),  # Increased embedding size for richer character features
    LSTM(256, return_sequences=True),  # Increased units for LSTM
    Dropout(0.2),  # Add dropout for regularization
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dropout(0.2),
    Dense(128, activation='relu'),  # Added Dense layer to add complexity
    Dense(vocab_size, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train with more epochs to let the model learn complex patterns
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Evaluate
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
3905/9375 ━━━━━━━━━━━━━━━━━━━━ 58s 11ms/step - accuracy: 0.2712 - loss: 2.5400

KeyboardInterrupt: 

In [ ]:
import numpy as np

def generate_text(model, seed_text, num_chars=150, temperature=0.5):
    generated_text = seed_text
    for _ in range(num_chars):
        input_seq = np.array([char_to_int[char] for char in seed_text]).reshape(1, -1)
        predictions = model.predict(input_seq, verbose=0)[0]

        # Adjust predictions by temperature
        predictions = np.log(predictions + 1e-10) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))

        # Sample the next character based on adjusted probabilities
        next_index = np.random.choice(len(predictions), p=predictions)
        next_char = int_to_char[next_index]

        generated_text += next_char
        seed_text = seed_text[1:] + next_char

    return generated_text



# Testing the generation function with a seed text
seed_text = text[:sequence_length]  # Starting text to kick off the prediction
generated_text = generate_text(model, seed_text)
print("Generated Text:")
print(generated_text)

Generated Text:
well, prince, so genoa and lucca are now just family estates of thebuonaparte went in the countess. but it was could be an austrian regiment was noticed the staff great respectful to the princess startled in the soldiers. she


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the pretrained GPT-2 model and tokenizer
model_name = "gpt2-xl"  #
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# gpt2-xl has 1.5 billion parameters and was trained on 40gb of txt data, 8 million diffrerent web pages.
# compare that to my model above with only 3 mb of text data. over 13000x smaller amount of training data.

# Set the start string
start_string = """
“Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes.”
"""

# Encode the input text for the model
input_ids = tokenizer.encode(start_string, return_tensors="pt")

# Generate text
output = model.generate(
    input_ids,
    max_length=300,  # Maximum length of the generated text
    num_return_sequences=1,  # Number of sequences to generate
    no_repeat_ngram_size=2,  # Prevents repeating phrases
    temperature=0.7,  # Controls creativity of output
    top_k=50,  # Limits sampling to top-k predictions
    top_p=0.9,  # Nucleus sampling
    do_sample=True  # Enables sampling
)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



“Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes.”

But the prince was not to be denied. He had been in the city a few days and had already
, through the use of his magic, been able to take possession of a
 isle in a nearby town.
"The Prince has been busy, but I've been trying to
. . . Oh, look, there's a dragon in front of me! I'll have to get
The prince, I guess!"
Carmilla's excitement was infectious. She was a very bright girl,
wonderful for the boys, and she was certainly going to do something
herself. It was no surprise that the young prince had the same
idea. Even though he was still very young, he had not been out of
studies for very long. Now that he knew what was out there, it was
possible that, when he grew up, his father would be able send him
on a mission to gather information. The boy was eager to know what it
took to become a hero, a great hero. At the moment, however, the only
information he could obtain was the information he got from hi